# **Гипотеза 1**

### Проверка гипотезы: "Рост процентных ставок ФРС негативно влияет на цену биткоина"

Мы хотим проверить, существует ли статистически значимая связь между изменением процентных ставок Федеральной резервной системы (ФРС) и изменением цены биткоина. В частности, мы проверяем гипотезу о том, что рост ставок ФРС негативно влияет на цену биткоина.

Для этого мы используем следующие методы:

- Корреляционный анализ — чтобы оценить силу и направление связи между ставкой ФРС и ценой биткоина. Мы предполагаем, что связь может быть линейной, и данные близки к нормальному распределению, поэтому используем корреляцию Пирсона.
- T-тест — чтобы проверить, различаются ли средние изменения цены биткоина до и после изменения ставки ФРС.
- Линейная регрессия — чтобы количественно оценить влияние изменения ставки ФРС на изменение цены биткоина.

Возьмем из нашего обработанных файл (`data.csv`) данные о времени, цене закрытия биткоина и ставках ФРС. Расчитаем изменение цены биткоина (`price_change`) и изменение ставки ФРС (`FRS_change`) для анализа их взаимосвязи. 



In [21]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, ttest_ind, zscore, iqr
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

btc_data = pd.read_csv('data_last.csv')
btc_data['timestamp'] = pd.to_datetime(btc_data['timestamp'])

df = btc_data[['timestamp', 'close', 'FRS_rate']]
df = df .sort_values(by='timestamp')
df['FRS_change'] = df['FRS_rate'].diff().fillna(0)
df["price_change"] = df["close"].diff()
df = df.dropna(subset=['price_change'])

### **Шаг 1. Корреляционный анализ**
1. Вычисляем коэффициент корреляции Пирсона между значениями ставки ФРС (FRS_rate) и ценой биткоина (close), что позволяет понять, существует ли линейная связь между текущим уровнем ставки ФРС и ценой биткоина.

2. Вычисляем коэффициент корреляции Пирсона между изменениями ставки ФРС (FRS_change) и изменениями цены биткоина (price_change), чтобы понять, существует ли лиейная связь между изменениями ставки ФРС и изменениями цены биткоина.

In [22]:
corr_rate_price, p_value = pearsonr(df["FRS_rate"].dropna(), df["close"].dropna())
print(f"Корреляция между ставкой ФРС и ценой биткоина: {corr_rate_price:.4f}, p-value: {p_value:.4f}")

# Корреляция между изменением ставки ФРС и изменением цены биткоина
corr_change_price, p_value = pearsonr(df["FRS_change"].dropna(), df["price_change"].dropna())
print(f"Корреляция между изменением ставки ФРС и изменением цены биткоина: {corr_change_price:.4f}, p-value: {p_value:.4f}")

Корреляция между ставкой ФРС и ценой биткоина: 0.2831, p-value: 0.0000
Корреляция между изменением ставки ФРС и изменением цены биткоина: -0.0008, p-value: 0.6763


1. Корреляция между ставкой ФРС и ценой биткоина

Корреляция: 0.2831 (слабая положительная связь).
p-value: 0.0000 (значимая связь, маловероятно, что она случайна).
Это может означать, что в периоды высоких ставок ФРС цена биткоина имеет тенденцию быть выше, но это не означает, что рост ставок ФРС вызывает рост биткоина.

2. Корреляция между изменением ставки ФРС и изменением цены биткоина

Корреляция: -0.0008 (почти нулевая связь).
p-value: 0.6763 (нет статистической значимости, т.е. связь случайна).
Это означает, что конкретные изменения ставки ФРС не приводят к предсказуемым изменениям цены биткоина.

**Шаг 2. Т-тест**

Разделяем данные на две группы: до и после изменения ставки ФРС.
Проводим T-тест для сравнения средних изменений цены биткоина в этих группах.

In [23]:
before_change = df[df["FRS_change"] == 0]["price_change"]
after_change = df[df["FRS_change"] != 0]["price_change"]

# T-тест на равенство средних
t_stat, p_val = ttest_ind(before_change.dropna(), after_change.dropna(), equal_var=False)
print(f"T-тест: t={t_stat:.4f}, p-value={p_val:.4f}")

T-тест: t=-1.0134, p-value=0.3219


_Так как p-value > 0.05, разница между изменениями цены до и после изменения ставок не значима._

### **Шаг 3. Регрессионный анализ**

Проведем дополнительно регрессионный анализ, используя метод наименьших квадратов

In [24]:
y = df["price_change"]

# Независимая переменная (изменение ставки ФРС)
X = df[["FRS_change"]]
X = sm.add_constant(X)  # Добавляем константу

# Построение модели
model = sm.OLS(y, X, missing="drop").fit()

# Вывод результатов регрессии
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           price_change   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1743
Date:                Tue, 18 Mar 2025   Prob (F-statistic):              0.676
Time:                        22:17:04   Log-Likelihood:            -1.5779e+06
No. Observations:              251934   AIC:                         3.156e+06
Df Residuals:                  251932   BIC:                         3.156e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2904      0.253      1.148      0.2

**Качество модели**

- R-squared (коэффициент детерминации): 0.000.

Это означает, что модель объясняет 0% изменчивости зависимой переменной (price_change), то есть, модель практически не объясняет изменения цены биткоина.

- Adj. R-squared (скорректированный R-squared): -0.000.

Скорректированный R-squared также близок к нулю, что подтверждает, что модель не улучшает объяснение изменчивости данных.

- F-statistic: 0.1743.

Низкое значение F-статистики и её p-value (0.676) указывают на то, что модель в целом не является статистически значимой.

**Коэффициенты модели**

- const (свободный член): 0.2904.

Это значение показывает ожидаемое изменение цены биткоина, если все независимые переменные равны нулю. Однако p-value (0.251) указывает на то, что этот коэффициент не является статистически значимым.

- FRS_change (изменение ставки ФРС): -21.0052.

Этот коэффициент показывает, что при увеличении изменения ставки ФРС на 1 единицу, изменение цены биткоина уменьшается на 21.0052 единицы. Однако p-value (0.676) указывает на то, что этот коэффициент не является статистически значимым.

**Дополнительные статистики**

- Omnibus и Jarque-Bera:

Эти тесты проверяют нормальность распределения остатков модели. Низкие p-values (0.000) указывают на то, что остатки не распределены нормально.

- Skew (асимметрия): -0.328.

Отрицательное значение указывает на левостороннюю асимметрию распределения остатков.

- Kurtosis (эксцесс): 50.086.

Высокое значение эксцесса указывает на наличие "тяжёлых хвостов" в распределении остатков, что также подтверждает отклонение от нормальности.

- Durbin-Watson: 2.008.

Значение близко к 2, что указывает на отсутствие автокорреляции в остатках.

**Выводы**

Модель не объясняет изменения цены биткоина: R-squared и Adj. R-squared близки к нулю, что указывает на отсутствие связи между изменением ставки ФРС и изменением цены биткоина.
Коэффициенты не значимы: Ни свободный член, ни коэффициент для FRS_change не являются статистически значимыми (p-values > 0.05).

### **Итог:**

Результаты регрессионного анализа показывают, что изменение ставки ФРС не оказывает статистически значимого влияния на изменение цены биткоина. При этом, в долгосрочной перспективе биткоин может быть выше в периоды высоких ставок (возможно, из-за других макроэкономических факторов). Рост ставок сам по себе не провоцирует резкое снижение биткоина, но его влияние может быть опосредованным через другие экономические процессы.

Следовательно, гипотеза отвергается.

# **Гипотеза 2**

### Проверка гипотезы: "Резкий рост объема торгов на Bitcoin за последние 3 часа предсказывает крупное движение цены в ближайшие 3 часа."

Возьмём данные за последние 365 дней, так как криптовалютный рынок динамично меняется, и на более длительном периоде рыночные показатели могу сильно отличаться друг от друга.

Для проверки этой гипотезы нам нужно сравнить две выборки:

Группа 1 (аномалии) – случаи, когда наблюдался резкий рост объема торгов.

Группа 2 (обычные периоды) – случаи, когда объем торгов оставался в нормальном диапазоне.

Для сравнения применяем t-тест, так как он позволяет проверить, есть ли статистически значимые различия в средних изменениях цены после аномалий и в обычные периоды, а также учитывает не только средние изменения, но и их разброс.

Если после аномального всплеска объема цена меняется сильнее, чем в обычные периоды, и если p-value < 0.05, то различие статистически значимо, и можно утверждать, что аномальный рост объема действительно предсказывает крупное движение цены.

Для определения аномалий используем 3 метода: Z-score, IQR и скользящее окно. Из визуализации видно, что аномалии, найденные с помощью Z-score и IQR совпадают. Метод скользящей средней даёт больше аномальных значений. Это связано с тем, что он, в отличие от остальных, учитывает локальные всплески (так как берем среднюю за 10 дней).

По графику цены закрытия можно заметить, что после аномалий скачки цены в среднем более существенные.


Для определения аномалий используем 3 метода: Z-score, IQR и скользящее окно. Из визуализации видно, что аномалии, найденные с помощью Z-score и IQR совпадают. Метод скользящей средней даёт больше аномальных значений. Это связано с тем, что он, в отличие от остальных, учитывает локальные всплески (так как берем среднюю за 10 дней).

По графику цены закрытия можно заметить, что после аномалий скачки цены в среднем более существенные.
Для t-теста будем использовать метод скользящего окна, так как оно учитывает не только глобальные выбросы, но и недавние изменения рынка. А это для нас важно, так как рынок криптовалют подвержен локальным всплескам объемов, связанным с новостями, ликвидациями и другими краткосрочными факторами.

In [25]:
df = btc_data
df.set_index('timestamp', inplace=True)
df['volume_3h'] = df['volume'].rolling(window='3H').sum()
df['volume_mean'] = df['volume'].rolling(window='10D').mean()
df['volume_var'] = df['volume'].rolling(window='10D').var()
df.dropna(inplace=True)


data_last_year = df.last('365D')

# Обнаружение аномальных всплесков объема

# Метод Z-score
data_last_year['volume_zscore'] = zscore(data_last_year['volume_3h'])
zscore_threshold = 3
data_last_year['volume_anomaly_zscore'] = data_last_year['volume_zscore'] > zscore_threshold

# Метод IQR
Q1 = data_last_year['volume_3h'].quantile(0.25)
Q3 = data_last_year['volume_3h'].quantile(0.75)
IQR = Q3 - Q1
iqr_threshold = 3
data_last_year['volume_anomaly_iqr'] = data_last_year['volume_3h'] > (Q3 + iqr_threshold * IQR)

# Метод скользящего окна
data_last_year['volume_anomaly_window'] = data_last_year['volume'] > (data_last_year['volume_mean'] + 3*np.sqrt(data_last_year['volume_var']))

# Анализ изменения цены через 3 часа (в долях)
data_last_year['price_change'] = abs((data_last_year['close'].shift(-12) - data_last_year['close'])/ data_last_year['close'])
data_last_year.dropna(inplace=True)

anomaly_price_changes = data_last_year[data_last_year['volume_anomaly_window']]['price_change']
normal_price_changes = data_last_year[~data_last_year['volume_anomaly_window']]['price_change']

# t-тест для проверки значимости
t_stat, p_value = ttest_ind(anomaly_price_changes, normal_price_changes)
print(f'Результат t-теста: t = {t_stat:.2f}, p = {p_value:.4f}')
if p_value < 0.05:
    print('Различие в изменениях цены статистически значимо.')
else:
    print('Различие в изменениях цены статистически не значимо.')

Результат t-теста: t = 21.12, p = 0.0000
Различие в изменениях цены статистически значимо.


Результат t-теста дал положительное значение t = 21.12, что свидетельствует о том, что среднее изменение цен при аномалиях выше, чем без них, причём довольно существенно. Также p-value около 0, из чего можно сделать вывод о том, что вероятность случайного возникновения наблюдаемой разницы между группами крайне мала (практически нулевая), а значит разница между группами статистически значима.

### **Вывод:** 
гипотеза не отвергается

# Гипотеза 3

Из визуализации мы заметили, что такие признаки, как стохастик и гистограмма MACD указывают на направление движения цены в следующие 15 минут.
Так же по графикам наиболее сильная корреляция наблюдается, если у нас либо оба признака находятся в категориях high, либо обра признака low.

Тогда сформулируем более точную гипотезу: 
### Цена в ближайшие 15 минут будет расти, если значения стохастика и MACD высокие, и будет падать, если низкие.

Для проверки оценим точность данного предоложения, посчитав долю случаев, когда цена действительно движется согласно нашей гипотезе.

In [34]:
conditions = [
    btc_data['Combined_Category'] == 'high / high',
    btc_data['Combined_Category'] == 'low / low'    
]
choices = [1, -1]

default = 0

btc_data['predict_direction'] = np.select(conditions, choices, default)

btc_data['direction'] = np.where(btc_data['target_change_15m_pct'] > 0, 1, -1)
btc_data['match'] = btc_data['predict_direction'] == btc_data['direction']

btc_data_filtered = btc_data[(btc_data['Combined_Category'] == 'high / high') | (btc_data['Combined_Category'] == 'low / low') ]
accuracy = btc_data_filtered['match'].mean()
print(accuracy)

0.8463593406003329


Видим довольно большое значение, значит, гипотеза не отвергается.
